<a href="https://colab.research.google.com/github/vishnu-chand/faceGyro/blob/main/faceGyro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
if '' or not os.path.exists('/content/pixUtils2'):
    !pip install youtube_dl mediapipe
    !rm -rf /content/sample_data
    !git clone https://github.com/vishnu-chand/pixUtils2.git;cd /content/pixUtils2;git checkout 410e006ce8a65c4391540977a6db0d7a7791b1b8
    !mkdir -p /content/db/srcVideos;cd /content/db/srcVideos;youtube-dl -f 'bestvideo[height<=480]+bestaudio/best[height<=480]' https://www.youtube.com/watch?v=lbwlVwNWLzU&t=12s&ab_channel=TheObamaWhiteHouse

!youtube-dl -f '[height<=480]' https://www.youtube.com/watch?v=lbwlVwNWLzU&t=12s&ab_channel=TheObamaWhiteHouse


%load_ext autoreload
%autoreload 2

%cd /content/pixUtils2
from pixUtils import *
%cd /content
import mediapipe as mp
from base64 import b64encode
from skimage.draw import disk
from IPython.display import HTML
import skimage.transform as trans
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates


def show_video(video_path):
    desPath = f'{dirname(video_path)}/smallSize.mp4'
    if not exists(desPath):
        !ffmpeg -y -i "{video_path}" -vf "scale=iw/2:ih/2" "{desPath}" > /dev/null 2>&1
    video_file = open(desPath, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video controls><source src="{video_url}"></video>""")

[youtube] lbwlVwNWLzU: Downloading webpage
[download] Weekly Address - The Honor of Serving You as President-lbwlVwNWLzU.mp4 has already been downloaded
[download] 100% of 9.35MiB
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/content/pixUtils2
/content


In [ ]:
tform = trans.SimilarityTransform()
face_mesh = mp.solutions.face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)


def getTransformMatrix(src, dst):
    '''
    https://docs.opencv.org/3.4/d4/d61/tutorial_warp_affine.html
    https://stackoverflow.com/questions/58026027/difference-between-similaritytransform-scikit-image-and-warpaffine-opencv
    https://stackoverflow.com/questions/25375597/what-happens-if-i-give-more-inputs-in-estimaterigidtransform-or-getaffinetransfo
    '''
    tform.estimate(src, dst)
    M = tform.params[0:2, :]
    return M


def getLandMark(img, pData):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.flags.writeable = False
    img_rows, img_cols, _ = img.shape
    results = face_mesh.process(img)
    datas = list()
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            lMarks = list()
            for idx, landmark in enumerate(face_landmarks.landmark):
                data = (np.nan, np.nan), (np.nan, np.nan)
                landmark_px = _normalized_to_pixel_coordinates(landmark.x, landmark.y, img_cols, img_rows)
                if landmark_px:
                    data = landmark_px
                lMarks.append(data)
            lMarks = np.array(lMarks)
            (x0, y0), (x1, y1) = lMarks.min(axis=0), lMarks.max(axis=0)
            box = x0, y0, x1 - x0, y1 - y0
            datas.append([box, lMarks])
    data = None
    if datas:
        if pData is None:
            data = max(datas, key=lambda x: x[0][2] * x[0][3])  # get the largest face using box area
        else:
            data = min(datas, key=lambda x: np.linalg.norm(x[1] - pData[1]))
    return data


def runWithDebug(vPath, startSec, desPath):
    historyWeight, dst, roiH, zoom = .5, None, None, .65
    leftEye = [249, 263, 362, 373, 374, 380, 381, 382, 384, 385, 386, 387, 388, 390, 398, 466]
    rightEye = [7, 33, 133, 144, 145, 153, 154, 155, 157, 158, 159, 160, 161, 163, 173, 246]
    mouth = [0, 13, 14, 17, 37, 39, 40, 61, 78, 80, 81, 82, 84, 87, 88, 91, 95, 146, 178, 181, 185, 191, 267, 269, 270, 291, 308, 310, 311, 312, 314, 317, 318, 321, 324, 375, 402, 405, 409, 415]
    cam = cv2.VideoCapture(vPath)
    pData = None
    with VideoWriter(desPath, cam) as vWriter:
        for fno, ftm, oImg in tqdm(videoPlayer(cam, startSec=startSec)):
            datas = getLandMark(oImg, pData)
            img = oImg.copy()
            if datas is not None:
                box, lMarks = datas
                roi = np.array([lMarks[r].mean(axis=0).astype('f4') for r in [leftEye, rightEye, mouth]])
                if roiH is None:
                    dst = roi.copy()
                    roiH = roi.copy()
                roiH = roiH * historyWeight + roi * (1 - historyWeight)
                img = img.astype('f4')
                for x, y in roiH:
                    rr, cc = disk((y, x), 10)
                    img[rr, cc] += 100
                img[img > 255] = 255
                img = cv2.warpAffine(img.astype('u1'), getTransformMatrix(roiH, dst * zoom), (-1, -1), borderValue=0)
                oImg2 = np.zeros_like(oImg)
                for x, y in lMarks:
                    oImg = cv2.circle(oImg, (int(x), int(y)), 2, (224, 0, 224), -1)
                    oImg2 = cv2.circle(oImg2, (int(x), int(y)), 2, (224, 0, 224), -1)
                putSubImg(img, imResize(oImg, scaleRC=.45), [-1, -1])
                putSubImg(img, imResize(oImg2, scaleRC=.45), [0, -1])
                # img = cv2.hconcat([img, cv2.vconcat([imResize(oImg, scaleRC=.5), imResize(oImg2, scaleRC=.5)])])
            # showImg('temp3_img', img, 1)
            pData = datas
            vWriter.write(img)


def run(vPath, startSec, desPath):
    historyWeight, dst, roiH, zoom = .5, None, None, .65
    leftEye = [249, 263, 362, 373, 374, 380, 381, 382, 384, 385, 386, 387, 388, 390, 398, 466]
    rightEye = [7, 33, 133, 144, 145, 153, 154, 155, 157, 158, 159, 160, 161, 163, 173, 246]
    mouth = [0, 13, 14, 17, 37, 39, 40, 61, 78, 80, 81, 82, 84, 87, 88, 91, 95, 146, 178, 181, 185, 191, 267, 269, 270, 291, 308, 310, 311, 312, 314, 317, 318, 321, 324, 375, 402, 405, 409, 415]
    cam = cv2.VideoCapture(vPath)
    pData = None
    with VideoWriter(desPath, cam) as vWriter:
        for fno, ftm, oImg in tqdm(videoPlayer(cam, startSec=startSec)):
            datas = getLandMark(oImg, pData)
            img = oImg.copy()
            if datas is not None:
                box, lMarks = datas
                roi = np.array([lMarks[r].mean(axis=0).astype('f4') for r in [leftEye, rightEye, mouth]])
                if roiH is None:
                    dst = roi.copy()
                    roiH = roi.copy()
                roiH = roiH * historyWeight + roi * (1 - historyWeight)
                img = cv2.warpAffine(img.astype('u1'), getTransformMatrix(roiH, dst * zoom), (-1, -1), borderValue=0)
            pData = datas
            vWriter.write(img)

srcPath = '/content/Weekly Address - The Honor of Serving You as President-lbwlVwNWLzU.mp4'
runWithDebug(srcPath, 0, f"{pathname(srcPath)}_des.mp4")



writing : /content/Weekly Address - The Honor of Serving You as President-lbwlVwNWLzU_des.mp4 @ 29.97002997002997 fps
6710it [02:02, 54.62it/s] 


In [ ]:
show_video(f"{pathname(srcPath)}_des.mp4")